# Natural Language Processing Demystified | Seq2Seq and Attention
https://nlpdemystified.org<br>
https://github.com/futuremojo/nlp-demystified<br><br>

Course module for this demo: https://www.nlpdemystified.org/course/seq2seq-and-attention

# Dependencias

In [1]:
import io
import json
import numpy as np
import pandas as pd
import random
import re
import tensorflow as tf
import unicodedata


from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("torch version:", torch.__version__)
print(f"Device name: '{torch.cuda.get_device_name()}'")
print("Device:", device)
print(f"Device properties: '{torch.cuda.get_device_properties(torch.cuda.current_device())}'")

torch version: 2.3.1
Device name: 'NVIDIA GeForce RTX 4060 Ti'
Device: cuda
Device properties: '_CudaDeviceProperties(name='NVIDIA GeForce RTX 4060 Ti', major=8, minor=9, total_memory=16059MB, multi_processor_count=34)'


## Dataset

In [3]:
!wget https://raw.githubusercontent.com/futuremojo/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_train.txt

--2024-09-16 03:39:52--  https://raw.githubusercontent.com/futuremojo/nlp-demystified/main/datasets/hun_eng_pairs/hun_eng_pairs_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5518306 (5.3M) [text/plain]
Saving to: ‘hun_eng_pairs_train.txt’

hun_eng_pairs_train 100%[===================>]   5.26M  14.3MB/s    in 0.4s    

2024-09-16 03:39:54 (14.3 MB/s) - ‘hun_eng_pairs_train.txt’ saved [5518306/5518306]



In [4]:
with open('hun_eng_pairs_train.txt') as file:
  train = [line.rstrip() for line in file]

Cada entrada consiste em uma sentença em hungaro seguida por uma sentença em inglês:

In [6]:
train[:3]

["Teszek rá, mit mondasz!<sep>I don't care what you say.",
 'Több olyan ember kell nekünk a csapatba, mint amilyen te vagy.<sep>We need more people like you on our team.',
 'Vigyázz a gyerekeimre!<sep>Take care of my children.']

Esse é um dataset relativamente pequeno para uma task de traduçãode linguagem. Mas vamos ver quão longe conseguimos ir com ele

In [7]:
len(train)

88647

Agora, vamos separar as sentenças em listas de acordo com os idiomas

In [8]:
SEPARATOR = '<sep>'
train_input, train_target = map(list, zip(*[pair.split(SEPARATOR) for pair in train]))

Como estamos lidando com uma linguagem de origem que usa caracteres acentuados, é importante aplicar a normalização Unicode.

No exemplo abaixo, dois conjuntos diferentes de Unicode produzem o mesmo caractere visualmente. O primeiro Unicode é para um 'a' acentuado, enquanto o segundo Unicode combina um 'a' com um acento.

In [12]:
print("\u00E1", "\u0061\u0301")

á á


Embora esses caracteres pareçam os mesmos para nós que os lemos, eles serão tratados de forma diferente por um modelo. Então, para evitar isso, a função a seguir normaliza quaisquer caracteres acentuados no mesmo conjunto de Unicode e, em seguida, os substitui por seus equivalentes ASCII.
https://docs.python.org/3/library/unicodedata.html

Aqui está um artigo informativo sobre a importância da normalização Unicode e como fazê-lo (incluindo o que NFD significa):
https://towardsdatascience.com/what-on-earth-is-unicode-normalization-56c005c55ad0

In [13]:
# Unicode normalization
def normalize_unicode(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')

Estamos construindo um modelo de tradução baseado em palavras , mas ainda queremos manter a pontuação e tratá-las como tokens separados, então inseriremos um espaço entre qualquer pontuação relevante e os caracteres ao redor delas. Dessa forma, nosso tokenizador (que não filtrará a pontuação) produzirá sinais de pontuação como tokens separados.

Esta função faz isso e a normalização unicode.

In [14]:
def preprocess_sentence(s):
  s = normalize_unicode(s)
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = s.strip()
  return s

In [15]:
# Preprocess both the source and target sentences.
train_preprocessed_input = [preprocess_sentence(s) for s in train_input]
train_preprocessed_target = [preprocess_sentence(s) for s in train_target]

Após o pré-processamento, o unicode deve ser normalizado e deve haver espaços em ambos os lados de qualquer pontuação.

In [16]:
train_preprocessed_input[:3]

['Teszek ra , mit mondasz !',
 'Tobb olyan ember kell nekunk a csapatba , mint amilyen te vagy .',
 'Vigyazz a gyerekeimre !']

Usaremos o **Teacher Forcing** com nosso modelo de tradução (especificamente, o decodificador). O primeiro passo dessa implementação consiste em colocar uma tag de início de frase ( \<sos\> ) e uma tag de fim de frase ( \<eos\> ) no início e no fim de cada frase-alvo, respectivamente.

In [17]:
def tag_target_sentences(sentences):
  tagged_sentences = map(lambda s: (' ').join(['<sos>', s, '<eos>']), sentences)
  return list(tagged_sentences)

In [19]:
train_tagged_preprocessed_target = tag_target_sentences(train_preprocessed_target)

In [20]:
train_tagged_preprocessed_target[:3]

["<sos> I don't care what you say . <eos>",
 '<sos> We need more people like you on our team . <eos>',
 '<sos> Take care of my children . <eos>']

A seguir, tokenizaremos nossas sentenças de entrada e de destino, tomando cuidado para manter a pontuação relevante.
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

Observe que também estamos incluindo um token fora do vocabulário (\<unk\>) na inicialização do Tokenizer. No momento da inferência, se o tokenizer encontrar uma palavra que não viu durante o ajuste inicial nos dados de treinamento, essa palavra será substituída por \<unk\> e o sistema de tradução precisará lidar com isso.

In [22]:
# Tokenizer for the Hungarian input sentences. Note how we're not filtering punctuation.
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
source_tokenizer.fit_on_texts(train_preprocessed_input)
# source_tokenizer.get_config()

In [23]:
source_vocab_size = len(source_tokenizer.word_index) + 1
print(source_vocab_size)

38539


In [24]:
# Tokenizer for the English target sentences.
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<unk>', filters='"#$%&()*+-/:;=@[\\]^_`{|}~\t\n')
target_tokenizer.fit_on_texts(train_tagged_preprocessed_target)
# target_tokenizer.get_config()

In [25]:
target_vocab_size = len(target_tokenizer.word_index) + 1
print(target_vocab_size)

10556


Em seguida, vetorizaremos as frases de entrada e de destino, assim como fizemos nas últimas demonstrações.
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [26]:
train_encoder_inputs = source_tokenizer.texts_to_sequences(train_preprocessed_input)